## Import necessary modules
Run this cell before running any other cells

In [1]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

# Printing and Logging
## Printing
You can use the **print()** function in Python to print messages to the screen. <br>
The message can be a string, or any other object, the object will be converted into a string before it is written to the screen. <br>

## Logging
You could use the logging module that is setup in *utils.py*. <br>
It prints to both your screen (standard output) as well as to log files (*ble.log*) in the *logs* directory. <br>
This is the recommended way to output messages, since the log files can help with debugging. <br>
The logging module also provides different log levels as shown below, each formatted with a different color for increased visibility. <br>

__**NOTE**__: You may notice that the DEBUG message is not printed to the screen but is printed in the log file. This is because the logging level for the screen is set to INFO and for the file is set to DEBUG. You can change the default log levels in *utils.py* (**STREAM_LOG_LEVEL** and **FILE_LOG_LEVEL**). 

## Formatting output
To format your strings, you may use %-formatting, str.format() or f-strings. <br>
The most "pythonic" way would be to use f-strings. [Here](https://realpython.com/python-f-strings/) is a good tutorial on f-strings. <br>

In [2]:
LOG.debug("debug")
LOG.info("info")
LOG.warning("warning")
LOG.error("error")
LOG.critical("critical")

2023-02-02 15:44:20,603 | INFO     |: info
2023-02-02 15:44:20,604 | WARNING  |: warning
2023-02-02 15:44:20,606 | ERROR    |: error
2023-02-02 15:44:20,608 | CRITICAL |: critical


<hr>

# BLE
## ArtemisBLEController
The class **ArtemisBLEController** (defined in *ble.py*) provides member functions to handle various BLE operations to send and receive data to/from the Artemis board, provided the accompanying Arduino sketch is running on the Artemis board. <br>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Functions</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reload_config()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reload changes made in <em>connection.yaml.</em></span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">connect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Connect to the Artemis board, whose MAC address is specified in <em>connection.yaml</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">disconnect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Disconnect from the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">is_connected()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Return a boolean indicating whether your controller is connected to the Artemis board or not.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">send_command(cmd_type, data)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Send the command <strong>cmd_type</strong> (integer) with <strong>data</strong> (string) to the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">receive_float(uuid) <br> receive_string(uuid) <br> receive_int(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Read the GATT characteristic (specified by its <strong>uuid</strong>) of type float, string or int. <br> The type of the GATT
            characteristic is determined by the classes BLEFloatCharacteristic, BLECStringCharacteristic or
            BLEIntCharacteristic in the Arduino sketch.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">start_notify(uuid, notification_handler)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Activate notifications on the GATT characteristic (specified by its <strong>uuid</strong>). <br> <strong>notification_handler</strong> is a
            function callback which must accept two inputs; the first will be a uuid string object and the second will
            be the bytearray of the characteristic value.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">bytearray_to_float(byte_array) <br> bytearray_to_string(byte_array) <br> bytearray_to_int(byte_array)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Convert the <strong>bytearray</strong> to float, string or int, respectively. <br> You may use these functions inside your
            notification callback function.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">stop_notify(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Stop notifications on the GATT characteristic (specified by its <strong>uuid</strong>).</span></th>
    </tr>
</table>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Variables</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">uuid</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A dictionary that stores the UUIDs of the various characteristics specified in <em>connection.yaml</em>.</span></th>
    </tr>
</table>

## Configuration
- The MAC address, Service UUID and GATT characteristic UUIDs are defined in the file: *connection.yaml*.
- They should match the UUIDs used in the Arduino sketch.
- The artemis board running the base code should display its MAC address in the serial monitor.
- Update the **artemis_address** in *connection.yaml*, accordingly.
- Make sure to call **ble.reload_config()** or **get_ble_controller()** (which internally calls **reload_config()**) after making any changes to your configuration file.

<hr>

In the below cell, we create an **ArtemisBLEController** object using **get_ble_controller()** (defined in *ble.py*), which creates and/or returns a single instance of **ArtemisBLEController**. <br>
<span style="color:rgb(240,50,50)"> __NOTE__: Do not use the class directly to instantiate an object. </span><br>

In [4]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2023-02-15 15:16:35,268 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:83:D5:66:86:3C
2023-02-15 15:16:37,789 | INFO     |: Connected to C0:83:D5:66:86:3C


## Receive data from the Artemis board

The cell below shows examples of reading different types (as defined in the Arduino sketch) of GATT characteristics.

In [5]:
# Read a float GATT Charactersistic
f = ble.receive_float(ble.uuid['RX_FLOAT'])
print(f)

TypeError: 'NoneType' object is not subscriptable

In [4]:
# Read a string GATT Charactersistic
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

[->9.0<-]


## Send a command to the Artemis board
Send the PING command and read the reply string from the string characteristic RX_STRING. <br>
__NOTE__: The **send_command()** essentially sends a string data to the GATT characteristic (TX_CMD_STRING). The GATT characteristic in the Arduino sketch is of type BLECStringCharacteristic.

In [5]:
ble.send_command(CMD.PING, "")

In [6]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

PONG


The cell below shows an example of the SEND_TWO_INTS command. <br> The two values in the **data** are separated by a delimiter "|". <br>
Refer Lab 2 documentation for more information on the command protocol.

In [7]:
ble.send_command(CMD.SEND_TWO_INTS, "2|-6")

The Artemis board should print the two integers to the serial monitor in the ArduinoIDE. 

## Disconnect

In [8]:
# Disconnect
ble.disconnect()

2023-02-14 14:14:37,788 | INFO     |: Disconnected from AB6C95E9-A4ED-4F7A-B208-0AFEAFDDA3AD


## Task 1

In [11]:
ble.send_command(CMD.ECHO, "Hello World!")

s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

Artemis said: Hello World!


## Task 2

In [13]:
ble.send_command(CMD.GET_TIME_MILLIS, "")

s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

T:2104284


## Task 3

In [14]:
def notif_callback(uuid, data):
    print((ble.bytearray_to_string(data)) + " is the received time")
    ble.stop_notify(ble.uuid['RX_STRING'])

ble.start_notify(ble.uuid['RX_STRING'], notif_callback)
ble.send_command(CMD.GET_TIME_MILLIS, "")

T:2582635 is the received time


## Task 4

In [15]:
def notif_callback(uuid, data):
    global instances
    pair = (ble.bytearray_to_string(data)).split('|')
    print("Sample " + str(instances) + " at " + pair[0] + " is " + pair[1])
    
    instances += 1
    if instances > 5:
        ble.stop_notify(ble.uuid['RX_STRING'])

instances = 1
ble.start_notify(ble.uuid['RX_STRING'], notif_callback)
ble.send_command(CMD.GET_TEMP_5s, "")

Sample 1 at T:2746942 is C:27.1
Sample 2 at T:2747943 is C:27.521
Sample 3 at T:2748943 is C:27.410
Sample 4 at T:2749943 is C:27.558
Sample 5 at T:2750943 is C:27.558


## Task 5

In [19]:
def notif_callback(uuid, data):
    pair = (ble.bytearray_to_string(data)).split('|')
    
    global instances
    print("Sample " + str(instances) + " at " + pair[0] + " is " + pair[1])
    instances += 1
    
    if round(time.time()*1000) - start_time > 5000:
        ble.stop_notify(ble.uuid['RX_STRING'])    

instances = 1
ble.start_notify(ble.uuid['RX_STRING'], notif_callback)
ble.send_command(CMD.GET_TEMP_5s_RAPID, "")
start_time = round(time.time()*1000)

Sample 1 at T:2983964 is C:27.669
Sample 2 at T:2983964 is C:27.521
Sample 3 at T:2983965 is C:27.298
Sample 4 at T:2983965 is C:27.743
Sample 5 at T:2984006 is C:27.706
Sample 6 at T:2984140 is C:27.595
Sample 7 at T:2984140 is C:28.40
Sample 8 at T:2984190 is C:27.669
Sample 9 at T:2984190 is C:27.447
Sample 10 at T:2984279 is C:27.743
Sample 11 at T:2984279 is C:28.263
Sample 12 at T:2984368 is C:27.595
Sample 13 at T:2984459 is C:27.892
Sample 14 at T:2984459 is C:27.966
Sample 15 at T:2984499 is C:27.632
Sample 16 at T:2984506 is C:27.669
Sample 17 at T:2984548 is C:27.669
Sample 18 at T:2984595 is C:27.818
Sample 19 at T:2984595 is C:27.743
Sample 20 at T:2984636 is C:27.743
Sample 21 at T:2984730 is C:28.40
Sample 22 at T:2984819 is C:27.298
Sample 23 at T:2984864 is C:27.632
Sample 24 at T:2984864 is C:28.226
Sample 25 at T:2984912 is C:27.335
Sample 26 at T:2984958 is C:28.40
Sample 27 at T:2984998 is C:27.595
Sample 28 at T:2985046 is C:27.743
Sample 29 at T:2985135 is C:27.1

In [55]:
ble.stop_notify(ble.uuid['RX_STRING'])